# This notebook is only used to create clean datasets and save them for other post

In [1]:
import time
import pandas as pd
import geopandas as gpd
import numpy as np

In [5]:
def get_data(wave, cols):

    df = pd.DataFrame()
    
    # get all distinct 2 first letters of columns
    cols_tag = [col[:2] for col in cols]

    for tag in cols_tag:
        path = f'../../../SHARE/data/sharew{wave}_rel8-0-0_ALL_datasets_stata/sharew{wave}_rel8-0-0_{tag}.dta'
        data = pd.read_stata(path)
        df = pd.concat([df, data], axis=1)

    # keep only columns of interest
    df = df[cols]

    # drop dupplicate columns
    df = df.loc[:,~df.columns.duplicated()]
    return df

def id_to_name(df):
    columns = {
        'language': 'Language',
        'br003_': 'YearCigarette',
        'country': 'Country',
        'dn014_': 'MaritalStatus',
        'dn042_': 'Gender',
        'dn003_': 'YearOfBirth',
        'br001_': 'Smoking',
        'br015_': 'SportFrequencyVigorous',
        'br016_': 'SportFrequencyModerate',
        'br039_': 'Alcool',
        'ph003_': 'HealthSelfPerception',
        'ph084_': 'Pain',
        'ph006d10': 'Cancer',
        'ph012_': 'Weight',
        'ph013_': 'Height',
        'ph041_': 'Glasses',
        'ph745_': 'HearingAid',
        'mh002_': 'Depression',
        'mh003_': 'Hopelessness',
        'mh004_': 'WishDeath',
        'mh005_': 'Guilt',
        'mh007_': 'SleepTrouble',
        'mh008_': 'Interest',
        'mh010_': 'Irritable',
        'mh011_': 'Appetite',
        'mh013_': 'Fatigue',
        'mh015_': 'ConcentrationReading',
        'mh016_': 'Enjoyment',
        'mh017_': 'Tears',
        'hc602_': 'MedVisit',
        'ex026_': 'Trustness'
        }
    
    column_types = {
        'YearOfBirth': 'float',
        'MaritalStatus': 'category',
        'Gender': 'category',
        'Smoking': 'category',
        'SportFrequencyVigorous': 'category',
        'SportFrequencyModerate': 'category',
        'HealthSelfPerception': 'category',
        'Pain': 'category',
        'Cancer': 'category',
        'Weight': 'float',
        'Height': 'float',
        'Glasses': 'category',
        'HearingAid': 'category',
        'Depression': 'category',
        'Hopelessness': 'category',
        'WishDeath': 'category',
        'Guilt': 'category',
        'SleepTrouble': 'category',
        'Interest': 'category',
        'Irritable': 'category',
        'Appetite': 'category',
        'Fatigue': 'category',
        'ConcentrationReading': 'category',
        'Enjoyment': 'category',
        'Tears': 'category',
        'YearOfBirth': 'float',
        'MedVisit': 'float',
        'Trustness': 'float'
    }

    for col in df.columns:
        df.rename(columns={col: columns[col]}, inplace=True)
        if col in column_types.keys():
            df[col] = df[col].astype(column_types[col])

    return df

def deal_nan(df, wave):
    values_to_replace = ["Don't know", "Refusal",
                     "Implausible value/suspected wrong", 
                     "Not codable", "Not answered",
                     "Not yet coded", "Not applicable"]
    df.replace(values_to_replace, float('NaN'), inplace=True)
    df['YearOfBirth'] = df['YearOfBirth'].astype(float)
    wave_to_year = {
            1: 2004,
            2: 2006,
            3: 2008,
            4: 2011,
            5: 2013,
            6: 2015,
            7: 2017,
            8: 2019
        }
    year = wave_to_year[wave]
    df["Age"] = year - df["YearOfBirth"]
    df = df.loc[df.Age >= 50,]
    return df

<br><br><br><br>

## Dataset with country, trustness and age

In [9]:
cols = ['country', 'ex026_', 'dn003_']
df = get_data(8, cols)
df = id_to_name(df)
df = deal_nan(df, 8)
df.to_csv('../../data/trustness_country_wave8.csv', index=False)